In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# Proxy configuration
PROXY = "127.0.0.1:24000"
# Path to chromedriver executable
PATH = "/opt/homebrew/Caskroom/chromedriver/121.0.6167.85/chromedriver-mac-arm64/chromedriver"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)

# player_ids = ['677950', '572233', '682998', '666150', '656896', '553993', '672515', '672695', '607054', '664983', '645444', '606466', '666971', '656976', '592626']
timeout = 25
# mlb_seasons = ['2023', '2022', '2021']
player_ids = [['dbacks', 'Alek Thomas', 'alek-thomas-677950']]


In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [ ]:
url = f"https://baseballsavant.mlb.com/savant-player/alek-thomas-677950?stats=statcast-r-hitting-mlb"
driver.get(url)

pitches = Select(driver.find_element(By.XPATH, r'//*[@id="pitchChartBreakdown"]'))
pitches.select_by_index(0)

handedness = Select(driver.find_element(By.XPATH, r'//*[@id="pitchChartHandedness"]'))
handedness.select_by_index(1)

headers_present = EC.visibility_of_element_located((By.XPATH, r'//*[@id="detailedPitches"]/tbody/tr[1]/td[2]/span'))
WebDriverWait(driver, timeout).until(headers_present)

src = driver.page_source
parser = BeautifulSoup(src, "lxml")
table = parser.find("div", attrs = {"id": "pitchTable"})

headers = table.findAll('th')
headerlist = [h.text.strip() for h in headers[:]]
headerlist.insert(0, "Name")
headerlist.insert(2, "Team")


In [ ]:
headerlist 

In [ ]:
for player in player_ids:
    url = f"https://baseballsavant.mlb.com/savant-player/{player[-1]}?stats=statcast-r-hitting-mlb"
    driver.get(url)

    pitches = Select(driver.find_element(By.XPATH, r'//*[@id="pitchChartBreakdown"]'))
    pitches.select_by_index(0)

    handedness = Select(driver.find_element(By.XPATH, r'//*[@id="pitchChartHandedness"]'))

    for hand in range(1, len(handedness.options)):
        handedness.select_by_index(hand)
        
        first_row_present = EC.visibility_of_element_located((By.XPATH, r'//*[@id="detailedPitches"]'))
        WebDriverWait(driver, timeout).until(first_row_present)

        src = driver.page_source
        parser = BeautifulSoup(src, "lxml")
        table = parser.find("div", attrs = {"id": "pitchTable"})
    
        rows = table.findAll('tr')[1:]
    
        player_stats = []
        for i in range(len(rows)):
            row_cells = []
            for td in rows[i].findAll('td'):
                row_cells.append(td.getText().strip())
            row_cells.insert(0, player[1])
            row_cells.insert(2, player[0])
            player_stats.append(row_cells)
    

In [ ]:
driver.quit()

In [ ]:
player_stats

In [ ]:
rows

In [ ]:
df = pd.DataFrame(player_stats, columns=headerlist)

In [ ]:
df

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


payload = {}
headers = {
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:122.0) Gecko/20100101 Firefox/122.0',
  'Accept': '*/*',
  'Accept-Language': 'en-US,en;q=0.5',
  'Accept-Encoding': 'gzip, deflate, br',
  'X-Requested-With': 'XMLHttpRequest',
  'Connection': 'keep-alive',
  'Referer': 'https://baseballsavant.mlb.com/savant-player/alek-thomas-677950?stats=statcast-r-hitting-mlb',
  'Cookie': '__cflb=02DiuJFcgbX2zpcHdRurfpjkBTD5BPySo6gPFqPfkQ556; AMCV_A65F776A5245B01B0A490D44%40AdobeOrg=1099438348%7CMCIDTS%7C19755%7CMCMID%7C08223760795725670020171971273569729942%7CMCAAMLH-1707411436%7C7%7CMCAAMB-1707411436%7C6G1ynYcLPuiQxYZrsz_pkqfLG9yMXBpb2zX5dvJdYQJzPXImdj0y%7CMCOPTOUT-1706813836s%7CNONE%7CMCAID%7CNONE%7CMCSYNCSOP%7C411-19762%7CvVersion%7C2.1.0; s_cc=true; s_sq=%5B%5BB%5D%5D; AMCVS_A65F776A5245B01B0A490D44%40AdobeOrg=1; s_vi=[CS]v1|32DDE6B6150B13BF-40000A6BE1F97D36[CE]; s_ecid=MCMID%7C08223760795725670020171971273569729942; _dd_s=rum=0&expire=1706807948264',
  'Sec-Fetch-Dest': 'empty',
  'Sec-Fetch-Mode': 'cors',
  'Sec-Fetch-Site': 'same-origin',
  'TE': 'trailers'
}


players = [['dbacks', 'Alek Thomas', '677950']]
handedness =['R', 'L']

In [2]:
for player in players:
    player_stats = []
    for hand in handedness:
        url = f"https://baseballsavant.mlb.com/player-services/statcast-pitches-breakdown?playerId={player[-1]}&position=8&hand={hand}&pitchBreakdown=pitches&timeFrame=yearly&season=&pitchType=&count=&updatePitches=true"

        r = requests.get(url, headers=headers)

        parser = BeautifulSoup(r.content, "html.parser")
        table = parser.find(id="detailedPitches")

        headersTH = table.find_all('th')
        headerlist = [h.text.strip() for h in headersTH[:]]
        headerlist.insert(0, "Name")
        headerlist.insert(2, "Team")
        headerlist.insert(3, "Hand") 
    
        rows = table.findAll('tr')[1:]
    
        for i in range(len(rows)):
            row_cells = []
            for td in rows[i].findAll('td'):
                row_cells.append(td.getText().strip())
            row_cells.insert(0, player[1])
            row_cells.insert(2, player[0])
            row_cells.insert(3, hand)
            player_stats.append(row_cells)

In [3]:
headerlist

['Name',
 'Year',
 'Team',
 'Hand',
 'Pitch Type',
 '#',
 '%',
 'PA',
 'AB',
 'H',
 '1B',
 '2B',
 '3B',
 'HR',
 'SO',
 'BBE',
 'BA',
 'XBA',
 'SLG',
 'XSLG',
 'WOBA',
 'XWOBA',
 'EV',
 'LA',
 'Whiff%',
 'PutAway%']

In [ ]:
player_stats

In [4]:
df = pd.DataFrame(player_stats, columns=headerlist)


In [5]:
df

,Name,Year,Team,Hand,Pitch Type,#,%,PA,AB,H,...,BA,XBA,SLG,XSLG,WOBA,XWOBA,EV,LA,Whiff%,PutAway%
0,Alek Thomas,2023,dbacks,R,4-Seam Fastball,333,28.5,83,74,25,...,.338,.302,.568,.509,.412,.378,90.0,9,22.0,12.6
1,Alek Thomas,2023,dbacks,R,Changeup,182,15.6,57,53,10,...,.189,.246,.245,.281,.198,.248,84.5,-7,24.2,21.2
2,Alek Thomas,2023,dbacks,R,Curveball,178,15.3,39,36,6,...,.167,.206,.167,.285,.189,.249,87.5,1,35.1,20.7
3,Alek Thomas,2023,dbacks,R,Slider,144,12.3,41,40,11,...,.275,.273,.375,.393,.289,.294,93.0,2,28.4,29.2
4,Alek Thomas,2023,dbacks,R,Sinker,141,12.1,38,37,7,...,.189,.234,.297,.384,.221,.274,90.4,-3,20.3,18.5
5,Alek Thomas,2023,dbacks,R,Cutter,104,8.9,28,26,9,...,.346,.345,.846,.614,.504,.426,88.9,9,20.4,8.0
6,Alek Thomas,2023,dbacks,R,Split Finger,38,3.3,12,12,3,...,.250,.251,.583,.295,.344,.238,91.2,-2,10.0,7.7
7,Alek Thomas,2023,dbacks,R,Sweeper,38,3.3,11,10,3,...,.300,.354,.300,.593,.307,.431,92.0,1,12.5,7.1
8,Alek Thomas,2023,dbacks,R,Forkball,5,0.4,2,2,0,...,.000,,.000,,.000,.000,,,66.7,66.7
9,Alek Thomas,2023,dbacks,R,Slurve,3,0.3,2,1,0,...,.000,,.000,,.363,.363,,,100.0,50.0
